# Monthly Update
October 2, 2017

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from datetime import date
import numpy as np
import os
import pandas as pd
import sys
ancestor = os.path.abspath(os.path.join(os.pardir, os.pardir))
if ancestor not in sys.path:
    sys.path.append(ancestor)
from Biblio_Reader import manager as mg
from Biblio_Reader.biblio_reader import pdf_utils
from Biblio_Reader.biblio_reader import pub_med
from Biblio_Reader.biblio_reader.text_tools import convertToText, text_tools
initial_date = date(2017, 3, 22)

Load existing data…

In [4]:
ytd = '../outputs/FCP_DATA_updated_{0}.csv'.format(date.today().year)
existing = pd.read_csv(ytd) if os.path.exists(ytd) else pd.read_csv('../outputs/FCP_DATA.csv')
if not os.path.exists(ytd):
    existing['DateAdded'] = initial_date
existing.dropna(axis=0, how='all', inplace=True)

Load today's data…

In [6]:
today = pd.read_csv(os.path.join('../outputs', '{0}.csv'.format(str(date.today()))))
today['DateAdded'] = date.today()

Lowercase, stripped titles for comparison…

In [7]:
today_lower = today.copy()
existing_lower = existing.copy()
today_lower['Title'] = today_lower['Title'].apply(lambda x: x.strip().lower())
existing_lower['Title'] = existing_lower['Title'].apply(lambda x: x.strip().lower())

Drop duplicates…

In [8]:
today_lower = today_lower.loc[~today_lower.duplicated(subset=['URL'])]  # internal dupes by URL
today_lower = today_lower.loc[~today_lower.duplicated(subset=['Title'])]  # internal dupes by Title
today_lower = today_lower.loc[~today_lower['URL'].isin(existing_lower['URL'])]  # cross-csv dupes by URL
today_lower = today_lower.loc[~today_lower['Title'].isin(existing_lower['Title'])]  # cross-csv dupes by Title
today = today[today['Unnamed: 0'].isin(today_lower['Unnamed: 0'])]

Visually check for straggling duplicates:

In [9]:
list(today['Title'].sort_values().unique())

['4D Multi-atlas Label Fusion Using Longitudinal Images',
 'A Multi-level Classification Framework for Multi-site Medical Data: Application to the ADHD-200 Collection',
 'ADHD subgroup discrimination with global connectivity features using hierarchical extreme learning machine: resting-state FMRI study',
 'Advances in studying brain morphology: The benefits of open-access data',
 'Age and sex modulate the variability of neural responses to naturalistic videos',
 'Anti-fragmentation of resting-state fMRI connectivity networks with node-wise thresholding',
 'Assessment of the impact of shared data on the scientific literature',
 'Classification based on neuroimaging data by tensor boosting',
 'Collective discovery of brain networks with unknown groups',
 'Data supporting publication" Probabilistic thresholding of functional connectomes: application to schizophrenia"',
 'Decreased structural connectivity and resting-state brain activity in the lateral occipital cortex is associated with s

See what preprints have since been published…

In [10]:
preprints_and_others = existing_lower[existing_lower['Journal Category'].isin(['Other', 'Preprint'])]
preprints = existing_lower[existing_lower['Journal Category'] == 'Preprint']
potentials = list(set(today_lower.loc[today_lower['Title'].isin(preprints_and_others['Title'])]['Title']
                ) | set(today_lower.loc[today_lower['Authors'].isin(preprints_and_others['Authors'])]['Title']))
print(potentials)

[]


In [11]:
today['i'] = range(max(existing['i']) + 1, max(existing['i']) + 1 + today.shape[0])
today = today.drop('Unnamed: 0', axis=1)
today.index = today.i

In [12]:
today_dir = os.path.abspath(os.path.join(ancestor, 'Biblio_Reader', str(date.today())))
if not os.path.exists(today_dir):
    os.makedirs(today_dir)

PubMed IDs:

In [16]:
today['PMCID'] = pub_med.get_ids(today)
today_pubmed = os.path.join(mg.WORKING_PATH, str(date.today()))
if not os.path.exists(today_pubmed):
    os.makedirs(today_pubmed)
pub_med.write_bib(today, today_pubmed)
pub_med.parse_bib(today_pubmed, "{0}_bibs.csv".format(today_pubmed))

In [19]:
d = pdf_utils.pdffinder(
    zip(today['i'], today['URL']),
    today_dir)

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


1571 https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5461345/


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


1585 http://books.google.com/books?hl=en&lr=&id=DYQ0DwAAQBAJ&oi=fnd&pg=PA362&dq=fcon_1000.projects.nitrc.org+OR+%22Rockland+Sample%22+OR+%221000+Functional+Connectomes%22+OR+%22International+Neuroimaging+Data-Sharing+Initiative%22+OR+%22Autism+Brain+Imaging+Data+Exchange%22+OR+ADHD-200+OR+%22Consortium+for+Reproducibility+and+Reliability%22&ots=iMogS0fZRB&sig=qkmHznLeZAdjQPXBgtVlUkZ7IEI
HTTP Error 404: Not Found 1588 http://www.sciencedirect.com/science/article/pii/S1053811917306912/pdfft?md5=bda7ab5972f30de9baa404123fb67631&pid=1-s2.0-S1053811917306912-main.pdf
1593 https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5517577/
PDFS FOUND: 5


In [20]:
d = pdf_utils.pdfopener(today, today_dir)

In [21]:
d = pdf_utils.arxiv_open(zip(today['i'], today['URL']),
    today_dir)

HTTP Error 404: Not Found 1560 https://arxiv.org/articles/10.3389/fnins.2017.00320/pdf.pdf
<urlopen error [Errno 8] nodename nor servname provided, or not known> 1560 https://arxiv.orghttps://www.frontiersin.org/articles/file/downloadfile/252093_supplementary-materials_presentations_1_pdf/octet-stream/Presentation 1.PDF/1/252093.pdf
HTTP Error 404: Not Found 1564 https://arxiv.org/content/early/2017/09/04/183814.full.pdf+html.pdf
HTTP Error 404: Not Found 1564 https://arxiv.org/panels_ajax_tab/biorxiv_tab_pdf/node:55759/1.pdf
<urlopen error [Errno 8] nodename nor servname provided, or not known> 1564 https://arxiv.orghttps://www.biorxiv.org/content/early/2017/09/04/183814.full.pdf.pdf


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


1571 https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5461345/


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


HTTP Error 404: Not Found 1574 https://arxiv.org/preprints/3010.pdf.pdf
<urlopen error [Errno 8] nodename nor servname provided, or not known> 1574 https://arxiv.orghttps://d3amtssd1tejdt.cloudfront.net/2017/3010/2/Kuhl2016fig_RightsLink_Printable_License.pdf.pdf
HTTP Error 404: Not Found 1574 https://arxiv.org/preprints/3010.pdf.pdf
HTTP Error 404: Not Found 1579 https://arxiv.org/content/early/2017/07/14/089060.full.pdf+html.pdf
HTTP Error 404: Not Found 1579 https://arxiv.org/panels_ajax_tab/biorxiv_tab_pdf/node:48685/1.pdf
<urlopen error [Errno 8] nodename nor servname provided, or not known> 1579 https://arxiv.orghttps://www.biorxiv.org/content/early/2017/07/14/089060.full.pdf.pdf
HTTP Error 404: Not Found 1582 https://arxiv.org/content/early/2017/08/17/177279.full.pdf+html.pdf
HTTP Error 404: Not Found 1582 https://arxiv.org/panels_ajax_tab/biorxiv_tab_pdf/node:53301/1.pdf
<urlopen error [Errno 8] nodename nor servname provided, or not known> 1582 https://arxiv.orghttps://www.bio

In [22]:
d = pdf_utils.plos_open(zip(today['i'], today['URL']),
    today_dir)

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


1571 https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5461345/


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


1585 http://books.google.com/books?hl=en&lr=&id=DYQ0DwAAQBAJ&oi=fnd&pg=PA362&dq=fcon_1000.projects.nitrc.org+OR+%22Rockland+Sample%22+OR+%221000+Functional+Connectomes%22+OR+%22International+Neuroimaging+Data-Sharing+Initiative%22+OR+%22Autism+Brain+Imaging+Data+Exchange%22+OR+ADHD-200+OR+%22Consortium+for+Reproducibility+and+Reliability%22&ots=iMogS0fZRB&sig=qkmHznLeZAdjQPXBgtVlUkZ7IEI
1593 https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5517577/


In [23]:
d = pdf_utils.liebert_open(zip(today['i'], today['URL']),
    today_dir)

unknown url type: '/articles/10.3389/fnins.2017.00320/pdf' 1560 /articles/10.3389/fnins.2017.00320/pdf
HTTP Error 400: Bad request 1560 https://www.frontiersin.org/articles/file/downloadfile/252093_supplementary-materials_presentations_1_pdf/octet-stream/Presentation 1.PDF/1/252093
unknown url type: '/content/early/2017/09/04/183814.full.pdf+html' 1564 /content/early/2017/09/04/183814.full.pdf+html
unknown url type: '/panels_ajax_tab/biorxiv_tab_pdf/node:55759/1' 1564 /panels_ajax_tab/biorxiv_tab_pdf/node:55759/1


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


1571 https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5461345/


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


unknown url type: '/preprints/3010.pdf' 1574 /preprints/3010.pdf
unknown url type: '/pdf/1707.05281' 1577 /pdf/1707.05281
unknown url type: '/content/early/2017/07/14/089060.full.pdf+html' 1579 /content/early/2017/07/14/089060.full.pdf+html
unknown url type: '/panels_ajax_tab/biorxiv_tab_pdf/node:48685/1' 1579 /panels_ajax_tab/biorxiv_tab_pdf/node:48685/1
unknown url type: '/content/early/2017/08/17/177279.full.pdf+html' 1582 /content/early/2017/08/17/177279.full.pdf+html
unknown url type: '/panels_ajax_tab/biorxiv_tab_pdf/node:53301/1' 1582 /panels_ajax_tab/biorxiv_tab_pdf/node:53301/1
unknown url type: '/pdf/1707.07932' 1583 /pdf/1707.07932
unknown url type: '/1099-4300/19/9/471/pdf' 1584 /1099-4300/19/9/471/pdf
1585 http://books.google.com/books?hl=en&lr=&id=DYQ0DwAAQBAJ&oi=fnd&pg=PA362&dq=fcon_1000.projects.nitrc.org+OR+%22Rockland+Sample%22+OR+%221000+Functional+Connectomes%22+OR+%22International+Neuroimaging+Data-Sharing+Initiative%22+OR+%22Autism+Brain+Imaging+Data+Exchange%22+O

In [24]:
d = pdf_utils.citeseer_open(zip(today['i'], today['URL']),
    today_dir)

HTTP Error 403: Forbidden 1571 https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5461345/
HTTP Error 401: Unauthorized 1585 http://books.google.com/books?hl=en&lr=&id=DYQ0DwAAQBAJ&oi=fnd&pg=PA362&dq=fcon_1000.projects.nitrc.org+OR+%22Rockland+Sample%22+OR+%221000+Functional+Connectomes%22+OR+%22International+Neuroimaging+Data-Sharing+Initiative%22+OR+%22Autism+Brain+Imaging+Data+Exchange%22+OR+ADHD-200+OR+%22Consortium+for+Reproducibility+and+Reliability%22&ots=iMogS0fZRB&sig=qkmHznLeZAdjQPXBgtVlUkZ7IEI
HTTP Error 403: Forbidden 1593 https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5517577/


Manually download as many of these as you can:

In [47]:
have_nots = set(pdf_utils.find_corrupted(today_dir))
haves = [int(i.rstrip('.pdf')) for i in os.listdir(today_dir) if i.endswith('pdf')]
have_nots = ({i for i in today['i'] if i not in haves} | have_nots)

In [50]:
for i in have_nots:
    path = today['URL']
    path = path.apply(lambda x: x[26:] if x.startswith('http://scholar.google.com/') else x)
    print(i, end=" : ")
    print(path[i], end=" : ")
    print(today['Title'][i])

1559 : https://uncch.pure.elsevier.com/en/publications/lcn-a-random-graph-mixture-model-for-community-detection-in-funct : LCN: a random graph mixture model for community detection in functional brain networks
1562 : http://ieeexplore.ieee.org/abstract/document/7950576/ : ADHD subgroup discrimination with global connectivity features using hierarchical extreme learning machine: resting-state FMRI study
1563 : http://journals.lww.com/neuroreport/Abstract/2017/08020/Examination_of_the_validity_of_the_atlas_informed.9.aspx : Examination of the validity of the atlas-informed approach to functional parcellation: a resting functional MRI study
1564 : https://www.biorxiv.org/content/early/2017/09/04/183814.abstract : Assessment of the impact of shared data on the scientific literature
1566 : https://link.springer.com/chapter/10.1007/978-3-319-67159-8_9 : FCNet: A Convolutional Neural Network for Calculating Functional Connectivity from functional MRI
1568 : http://link.springer.com/chapter/10

Check again…

In [60]:
have_nots = set(pdf_utils.find_corrupted(today_dir))
haves = [int(i.rstrip('.pdf')) for i in os.listdir(today_dir) if i.endswith('pdf')]
have_nots = ({i for i in today['i'] if i not in haves} | have_nots)

In [61]:
for i in have_nots:
    path = today['URL']
    path = path.apply(
           lambda x:
               x[26:] if x.startswith('http://scholar.google.com/') else x
           )
    print(i, end=" : ")
    print(path[i], end=" : ")
    print(today['Title'][i])

1559 : https://uncch.pure.elsevier.com/en/publications/lcn-a-random-graph-mixture-model-for-community-detection-in-funct : LCN: a random graph mixture model for community detection in functional brain networks


After downloading pdfs, convert to plaintext:

In [89]:
txt_dir = os.path.join(today_dir, 'txt')
if not os.path.exists(txt_dir):
    os.makedirs(txt_dir)
convertToText.walkAndText(today_dir, txt_dir)

/Users/jon.clucas/Biblio_Reader/2017-10-03/1560.pdf
/Users/jon.clucas/Biblio_Reader/2017-10-03/1561.pdf
/Users/jon.clucas/Biblio_Reader/2017-10-03/1562.pdf
/Users/jon.clucas/Biblio_Reader/2017-10-03/1563.pdf
/Users/jon.clucas/Biblio_Reader/2017-10-03/1564.pdf
/Users/jon.clucas/Biblio_Reader/2017-10-03/1565.pdf
/Users/jon.clucas/Biblio_Reader/2017-10-03/1566.pdf
/Users/jon.clucas/Biblio_Reader/2017-10-03/1567.pdf
/Users/jon.clucas/Biblio_Reader/2017-10-03/1568.pdf
/Users/jon.clucas/Biblio_Reader/2017-10-03/1569.pdf
/Users/jon.clucas/Biblio_Reader/2017-10-03/1570.pdf
/Users/jon.clucas/Biblio_Reader/2017-10-03/1571.pdf
/Users/jon.clucas/Biblio_Reader/2017-10-03/1572.pdf
/Users/jon.clucas/Biblio_Reader/2017-10-03/1573.pdf
/Users/jon.clucas/Biblio_Reader/2017-10-03/1574.pdf
/Users/jon.clucas/Biblio_Reader/2017-10-03/1575.pdf
/Users/jon.clucas/Biblio_Reader/2017-10-03/1576.pdf
/Users/jon.clucas/Biblio_Reader/2017-10-03/1577.pdf
/Users/jon.clucas/Biblio_Reader/2017-10-03/1578.pdf
/Users/jon.c

Collect relevant ¶s:

In [121]:
today['Excerpt'] = pd.Series(
                       text_tools.find_paragraphs(
                           txt_dir,
                           [*mg.WEIGHTED_SETS, *mg.UNWEIGHTED_SETS],
                           os.path.join(today_dir, 'excerpts')
                       )
                   ).apply(lambda x: "\n\n".join(x))

In [122]:
sets = text_tools.assoc_sets(
            today,
            txt_dir,
            mg.WEIGHTED_SETS,
            mg.UNWEIGHTED_SETS
        )
today['Sets'] = pd.Series(
    {k:sets[k]
        for k in sets if k>=min(today['i'])
    }
)

Update csv:

In [124]:
existing.merge(today, how='outer').to_csv(ytd)

Vet what's left on the web:

In [140]:
pdf_utils.find_articles_left(
    zip(today['i'].astype(str), today['URL']),
    txt_dir)

1597 https://uncch.pure.elsevier.com/en/publications/lcn-a-random-graph-mixture-model-for-community-detection-in-funct


[('1597',
  'https://uncch.pure.elsevier.com/en/publications/lcn-a-random-graph-mixture-model-for-community-detection-in-funct')]

Human-readable excerpt txt file:

In [165]:
with open(os.path.join(today_dir, 'excerpts.txt'), 'w') as f:
    f.write("".join(["{0}\n\n{1}\n\n\n\n".format(i, j)
        for i, j in zip(today.index, today['Excerpt'])])
    )

Come back after all the manual vetting and run the numbers!

In [5]:
existing.columns

Index(['Unnamed: 0', 'i', 'Title', 'URL', 'Year', 'Citations', 'Versions',
       'Cluster ID', 'PDF link', 'Citations list', 'Versions list',
       'Citation link', 'Excerpt', 'Journal', 'Authors', 'Publisher',
       'Citations Per Year', 'Journal Category', 'PMCID', 'Affiliations',
       'Qualifiers', 'Data Use', 'Sets', 'Contributor', 'Duplicate of',
       'DateAdded'],
      dtype='object')

In [27]:
existing.shape

(1598, 26)

In [43]:
len(existing.loc[
    (existing['Data Use'].isin(['Y', 'N'])) &
    (existing['Contributor'] == 'Contributor')
])

153

In [41]:
len(existing.loc[
    (existing['Journal Category'] == 'Journal') & 
    (existing['Data Use'].isin(['Y'])) &
    (existing['Contributor'] == 'Contributor')
])

77

In [34]:
len(existing.loc[
    (existing['Data Use'].isin(['Y', 'N'])) &
    (existing['Contributor'] == 'Not a Contributor')
])

1289

In [44]:
len(existing.loc[
    (existing['Journal Category'] == 'Journal') & 
    (existing['Data Use'].isin(['Y'])) &
    (existing['Contributor'] == 'Not a Contributor')
])

547

In [31]:
len(existing.loc[
    (existing['Data Use'].isin(['Y', 'N']))
])

1442

In [38]:
len(existing.loc[    
    (existing['Journal Category'] == 'Journal') & 
    (existing['Data Use'].isin(['Y', 'N']))
])

993